# Text Quality Analyzer
by Ricky | v1(20/04/2020)

## Observations

Are interpreted as wrong words:
- URLs
- alphanumeric codes (ex: dsa543nqhf7dh3ndsaku43878)

Are interpreted as w:
- palabras en femenino
- palabras en plural
- conjugaciones verbales
- palabras acompañadas de símbolos o puntuación en su inicio o fin (ej: "ejemplo:")

Son omitidas:
- Todas las expresiones que solo contienen números y símbolos (ej: 19/04/2020, 12.345.678-9)


## Posibles Mejoras:

Alamacenar las palabras erroneas para analizar si en verdad lo son o son tecnisismo, abreviaciones o términos de la cultura de la empresa.

Detectar el idioma de un texto (para no deleccionar todas sus palabras como erroneas)


## Instalación de Librerías

In [1]:
# !pip install datetime

## Importaciones

In [2]:
from os import listdir
from functools import reduce
from os.path import join as pjoin
# from googletrans import Translator
# import goslate

from datetime import datetime as dt


## Variables

In [3]:
# Sybols to delete before and after the word:
NOT_ALLOWED = ' \n.,-:;?¿\'"'

## Funciones

In [4]:
def file_to_words_list(file_path):
    '''
    Receives the path of a file. Each line of the file contains one word.
    Return a list with the words.
    '''
    with open(file_path, 'r', encoding='utf-8') as file:
        return list(map(lambda word: word.strip(), file.readlines()))


def files_to_lists(directory, *files):
    '''
    Receives a directory. If file names are given, those will be processed.
    Otherwise all the files in the directory will be processed.
    Return a list of lists with words.
    '''
    if files: lst_files = files # specified files are processed
    else: lst_files = listdir(directory) # all files in directory are processed
    return list(map(lambda x: file_to_words_list(pjoin(directory, x)), lst_files))

In [5]:
def has_alpha_char(word):
    '''
    If word contains alphabet characters returns True. False otherwise.
    '''
    try:
        return reduce(lambda a,b: a or b, map(lambda x: x.isalpha(), word))
    except Exception:
        print(f'No se pueden revisar los carácteres de la palabra: "{word}"')
        return False

## Clase Principal

In [6]:
class TextQualityAnalyzer:
    '''
    This class checks if the word exists in the provided words lists. 
    '''
    
    def __init__(self, *words_lists):
        self.idiom_words = reduce(lambda lst1, lst2: lst1 + lst2, words_lists)
        self.text = ''
        self.__iter_words = None
        self.total_words = None
        self.c_wrong_words = None

# Este fragmento elimina símbolos en medio de una palabra, lo que sería un error.
#     def __clean_word(self, word):
#         return ''.join(filter(lambda char: char.isalpha(), word)) 
    
    def __parse_text(self):
#         self.__iter_words = map(self.__clean_word, self.text.split(' '))
         self.__iter_words = map(lambda word: word.strip(NOT_ALLOWED), self.text.split(' '))
    
    def word_exist(self, word):
        '''
        If the word exist in the word list return True. False otherwise.
        '''
        word = word.lower()
        return word in self.idiom_words
        
    def __check_words_existance(self):
        for word in self.__iter_words:
            if has_alpha_char(word):
                self.c_wrong_words += int(not self.word_exist(word))
                self.total_words += 1
            
    def analize(self, text):
        '''
        Receives a text and return a score(int) depending of the existing words.
        '''
        self.total_words = 0
        self.c_wrong_words = 0
        
        self.text = text
        self.__parse_text()
        self.__check_words_existance()
        return self.total_words // (self.c_wrong_words + 1)


## Ejemplos

In [7]:
# Examples imports:
import pandas as pd
import boto3
from io import BytesIO

In [8]:
# Variables:
DIRECTORY_DICTS = 'languages_dictionaries'
SPANISH_DICT = 'spanish.0'
ENGLISH_DICT = 'english.0'


### Creación de Objeto TextQualityAnalyzer con Archivo de Palabras en Español

In [9]:
start = dt.now()

lst_words = file_to_words_list(pjoin(DIRECTORY_DICTS, SPANISH_DICT)) #Change here the language
tqa = TextQualityAnalyzer(lst_words) # Here you can add all lists you want separated by ','

print(f'tiempo ejecución: {dt.now() - start}')

tiempo ejecución: 0:00:00.505523


### Ejemplo de Análisis y Conteos de Palabras:

In [10]:
tqa.analize('mí número personal: 12.345.678-9 fsafdas')

2

In [11]:
tqa.total_words

4

In [12]:
tqa.c_wrong_words

1

### Ejemplo con DataFrame

In [13]:
# Variables to connect AWS S3

AWS_REGION = 'us-east-1'
# MIN_SENTENCE_LENGTH_IN_CHARS = 10
# MAX_SENTENCE_LENGTH_IN_CHARS = 4500
# COMPREHEND_BATCH_SIZE = 25  ## This batch size results in groups no larger than 25 items
# NUMBER_OF_BATCHES = 30
# ROW_LIMIT = 10000

s3 = boto3.client('s3')
comprehend_client = boto3.client('comprehend',region_name = AWS_REGION)

In [14]:
# Variables
bucket = "sura-text-mining-poc"
key = 'raw/ratings/ratings.csv'
key2 = 'raw/complaints/complaints.csv'

# Creation of first dataframe
obj = s3.get_object(Bucket=bucket, Key=key)
obj = BytesIO(obj['Body'].read())

df = pd.read_csv(obj, sep=",", error_bad_lines=False, warn_bad_lines=False, nrows=7)
df.head(3)

,author,posted_on,rating,text
0,"Alantae of Chesterfeild, MI","Nov. 22, 2016",1,I used to love Comcast. Until all these consta...
1,"Vera of Philadelphia, PA","Nov. 19, 2016",1,I'm so over Comcast! The worst internet provid...
2,"Sarah of Rancho Cordova, CA","Nov. 17, 2016",1,If I could give them a negative star or no sta...


In [15]:
# Creation of second dataframe:
obj2 = s3.get_object(Bucket=bucket, Key=key2)
obj2 = BytesIO(obj2['Body'].read())
# obj = gzip.GzipFile(fileobj=obj)

df2 = pd.read_csv(obj2, sep=",", error_bad_lines=False, warn_bad_lines=False, nrows=7)
df2.head(3)

,Ticket #,Customer Complaint,Date,Time,Received Via,City,State,Zip code,Status,Filing on Behalf of Someone,Description
0,250635,Comcast Cable Internet Speeds,4/22/2015,3:53:50 PM,Internet,Abingdon,Maryland,21009,Closed,No,I have been contacting Comcast Internet Techni...
1,223441,Payment disappear - service got disconnected,4/8/2015,10:22:56 AM,Internet,Acworth,Georgia,30102,Closed,No,Back in January 2015 I made 2 payments: One fo...
2,242732,Speed and Service,4/18/2015,9:55:47 AM,Internet,Acworth,Georgia,30101,Closed,Yes,Our home is located at in Acworth Georgia 3010...


In [16]:
# Selection of columns:
df_example = df[['author', 'posted_on', 'text']]
df_example.columns = ['author', 'date', 'text']
df_example.shape

(7, 3)

In [17]:
# Spanish translations:

lst_translations= ['Me encantaba Comcast. Hasta todas estas actualizaciones constantes. Mi Internet y mi cable se bloquean mucho por la noche y, a veces, durante el día, algunos canales ni siquiera funcionan y, a pedido, a veces tampoco funcionan. Deseo que hagan algo al respecto. Porque hace solo unos minutos, Internet se ha bloqueado durante unos 20 minutos sin ningún motivo. Estoy cansado de eso y estoy pensando en cambiar a Wow o algo así. Por favor, no obtengas Xfinity.',
 'Estoy tan cansado de Comcast! El peor proveedor de internet. Estoy tomando clases en línea y muchas veces llegué tarde con mis tareas debido a las interrupciones de energía en mi área que conducen a un servicio de Internet de baja calidad. Definitivamente cambiando a Verizon. Prefiero pagar $ 10 adicionales que tratar con problemas de Internet Comcast y sin parar.',
 'Si pudiera darles una estrella negativa o ninguna estrella en esta crítica, lo haría. Nunca he trabajado con ninguna industria con tan mal servicio al cliente como Comcast. No es una cuestión de dinero porque hago lo suficiente por encima y más allá para pagar sus servicios, pero son una estafa legítima. Creo que son la estafa más grande desde el colapso principal de la industria hipotecaria y espero mudarme a algún lugar donde Comcast no exista. El hecho de querer ayudar o hacer lo correcto es sinceramente asombroso. Si tiene que llamar, lo que hace PARA TODOS LOS PROBLEMAS: facturación, conexión / servicio, agregar o eliminar servicios, errores, no importa que se le transfiera un mínimo de 4 veces. Todos dicen lo mismo y pasan los problemas a la siguiente persona y nadie resuelve el problema. Ofrecen paquetes promocionales en plazos pequeños y nunca pueden acceder a ellos nuevamente para que luego lo actualicen sin que lo desee y cambien su facturación. Han pasado 5 meses y me han cobrado $ 40 al mes desde que comencé con ellos. La grosería descarada que debe hacerlo calificado para hacer este trabajo es el tipo de servicio de calidad que le otorga esta revisión. Entonces ... Querido Comcast, apestas. Sinceramente, un cliente que no puede esperar para nunca volver a usar su servicio.',
 'He tenido las peores experiencias hasta ahora desde la instalación el 10/10/16. Nada más que problemas. Dos no presentaciones en citas de servicio programadas, dificultad extrema para agregar cuadros al segundo piso. ¿Qué es tan difícil de agregar cuadros a una cuenta existente? ¡No, gracias, no estoy comenzando una segunda cuenta para el segundo piso de la misma casa! ¿Un paquete de paquete separado? Todo lo que quería era agregar algunas cajas. Aparentemente esto no es posible. Bueno, supongo que no es posible seguir siendo cliente.',
 'Revise su contrato cuando se registre en Comcast ya que sus ofertas anunciadas no coinciden con el contrato que emiten. Me inscribí por $ 49.99 a 150Mbps de internet por 2 años, sin embargo mi contrato tiene $ 19.99 por 25Mbps de internet por 2 años. ¡Dicen que hay un complemento por $ 30 que lo impulsa a Blast! Pro, sin embargo, esto no es parte del contrato, lo que significa que Comcast puede aumentar el precio cuando lo desee dentro de los 2 años. Esto significa que no he recibido la tarifa anunciada. Hasta el momento, Comcast se ha negado a emitir un contrato corregido, o emitir por escrito que los $ 30 permanecerán a ese precio durante 2 años. Solo tengo que confiar en ellos. Así que cuidado, Comcast está haciendo las prácticas ilegales habituales, supongo que atraparán a la gente y espero que no se den cuenta y terminen pagando más de lo que deberían.',
 'Gracias a Dios. Estoy cambiando a plato. Me dieron precios increíbles y super personas con las que tratar. Realmente puedes entender lo que están diciendo. Estoy muy emocionado de finalmente poder devolver este equipo, aunque todavía no he recibido la seguridad del hogar como lo prometí 4 veces. Vaya a h * ll Comcast. Me has hecho miserable y me has hecho perder muchas horas de trabajo con tus promesas.',
 'He sido cliente por mucho tiempo y solo tengo a Xfinity como mi ISP por un tiempo. Mientras estaba en el Walmart local el 4 de noviembre de 2016, había representantes de clientes de Xfinity que realizaban promociones para y en el área de Salt Lake City. Hablé con un representante y pude obtener y firmar un contrato para Pro Blast a $ 50.00 al mes sin contrato ni cargos por cancelación anticipada. Recibí un correo electrónico de Xfinity indicando los cambios que se realizarían en mi cuenta. Declaró que no solo estaría bajo contrato por 24 meses sino que habría cargos por cancelación anticipada. Esto no es para lo que me había inscrito originalmente y lo indica específicamente en el contrato que firmé. Me contacté con el servicio al cliente de Xfinity y me dijeron que no podían ver el contrato por teléfono que tendría que ir a la tienda de Xfinity en persona. Fui a la tienda Xfinity el 8 de noviembre de 2016 y me dijeron que estaría bajo contrato y que no había forma de evitarlo. Debido a esto, he cancelado la actualización y volví a mi plan original. Es simple y llanamente. Cuando se firma un contrato, debe cumplirse con lo que se indica en él. Xfinity es deshonesto y no confiable. Por lo tanto, buscaré y cambiaré mi ISP lo antes posible a otra compañía. Xfinity no merece un cheque de mi parte ni de nadie más que yo conozca.']

In [18]:
len(lst_translations)

7

In [19]:
# Add translations to DF:
df_example['spanish_text'] = lst_translations
df_example.head(3)

,author,date,text,spanish_text
0,"Alantae of Chesterfeild, MI","Nov. 22, 2016",I used to love Comcast. Until all these consta...,Me encantaba Comcast. Hasta todas estas actual...
1,"Vera of Philadelphia, PA","Nov. 19, 2016",I'm so over Comcast! The worst internet provid...,Estoy tan cansado de Comcast! El peor proveedo...
2,"Sarah of Rancho Cordova, CA","Nov. 17, 2016",If I could give them a negative star or no sta...,Si pudiera darles una estrella negativa o ning...


In [20]:
# Executions of the analysis and creation of DF columns:

analysis = [(tqa.analize(x), tqa.total_words,tqa.c_wrong_words) for x in df_example['spanish_text']]
print(analysis)
df_example['total_words'] = [x[1] for x in analysis]
df_example['wrong_words'] = [x[2] for x in analysis]
df_example['priority'] = [x[0] for x in analysis]
df_example

No se pueden revisar los carácteres de la palabra: ""
[(19, 77, 3), (14, 59, 3), (54, 216, 3), (28, 85, 2), (16, 146, 8), (13, 68, 4), (14, 239, 15)]


,author,date,text,spanish_text,total_words,wrong_words,priority
0,"Alantae of Chesterfeild, MI","Nov. 22, 2016",I used to love Comcast. Until all these consta...,Me encantaba Comcast. Hasta todas estas actual...,77,3,19
1,"Vera of Philadelphia, PA","Nov. 19, 2016",I'm so over Comcast! The worst internet provid...,Estoy tan cansado de Comcast! El peor proveedo...,59,3,14
2,"Sarah of Rancho Cordova, CA","Nov. 17, 2016",If I could give them a negative star or no sta...,Si pudiera darles una estrella negativa o ning...,216,3,54
3,"Dennis of Manchester, NH","Nov. 16, 2016",I've had the worst experiences so far since in...,He tenido las peores experiencias hasta ahora ...,85,2,28
4,"Ryan of Bellevue, WA","Nov. 14, 2016",Check your contract when you sign up for Comca...,Revise su contrato cuando se registre en Comca...,146,8,16
5,"Terri of Mobile, AL","Nov. 9, 2016",Thank God. I am changing to Dish. They gave me...,Gracias a Dios. Estoy cambiando a plato. Me di...,68,4,13
6,"Kellie of Salt Lake City, UT","Nov. 9, 2016",I Have been a long time customer and only have...,He sido cliente por mucho tiempo y solo tengo ...,239,15,14


In [21]:
# DF sorted by descending priority score:
df_example.sort_values('priority', ascending=False)

,author,date,text,spanish_text,total_words,wrong_words,priority
2,"Sarah of Rancho Cordova, CA","Nov. 17, 2016",If I could give them a negative star or no sta...,Si pudiera darles una estrella negativa o ning...,216,3,54
3,"Dennis of Manchester, NH","Nov. 16, 2016",I've had the worst experiences so far since in...,He tenido las peores experiencias hasta ahora ...,85,2,28
0,"Alantae of Chesterfeild, MI","Nov. 22, 2016",I used to love Comcast. Until all these consta...,Me encantaba Comcast. Hasta todas estas actual...,77,3,19
4,"Ryan of Bellevue, WA","Nov. 14, 2016",Check your contract when you sign up for Comca...,Revise su contrato cuando se registre en Comca...,146,8,16
1,"Vera of Philadelphia, PA","Nov. 19, 2016",I'm so over Comcast! The worst internet provid...,Estoy tan cansado de Comcast! El peor proveedo...,59,3,14
6,"Kellie of Salt Lake City, UT","Nov. 9, 2016",I Have been a long time customer and only have...,He sido cliente por mucho tiempo y solo tengo ...,239,15,14
5,"Terri of Mobile, AL","Nov. 9, 2016",Thank God. I am changing to Dish. They gave me...,Gracias a Dios. Estoy cambiando a plato. Me di...,68,4,13


### Scripts que Generan Traducciones

In [22]:
# # Generation of translations:



# translator = goslate.Goslate()

# df_example['spanish_text'] = [translator.translate(text, 'es') for text in df_example.text]
# df_example.head(3)


In [23]:
# text = "I used to love Comcast. Until all these constant updates. My internet and cable crash a lot at night, and sometimes during the day, some channels don't even work and on demand sometimes don't play either. I wish they will do something about it. Because just a few mins ago, the internet have crashed for about 20 mins for no reason. I'm tired of it and thinking about switching to Wow or something. Please do not get Xfinity."
# # translator.translate(text, src='en', dest='es')

# gs = goslate.Goslate()
# translatedText = gs.translate(text,'es')

# print(translatedText)


In [24]:
# import goslate

# text = "Hello World"

# gs = goslate.Goslate()
# translatedText = gs.translate(text,'es')

# print(translatedText)

# Intentos Fallidos

Librerías externas como pyenchant, nltk

## pyenchant

In [25]:
# !pip install -U pip

In [26]:
# !pip install pyenchant

In [27]:
# import enchant
# d = enchant.Dict("en_US")
# d.check("Hello")

In [28]:
import os
os.system('uname -o')

0

In [29]:
from sys import platform as plat
plat

'linux'

In [30]:
import platform
platform.linux_distribution

<function platform.linux_distribution(distname='', version='', id='', supported_dists=('SuSE', 'debian', 'fedora', 'redhat', 'centos', 'mandrake', 'mandriva', 'rocks', 'slackware', 'yellowdog', 'gentoo', 'UnitedLinux', 'turbolinux', 'arch', 'mageia'), full_distribution_name=1)>

In [31]:
# !sudo yum update -y

In [32]:
# !sudo yum install -y libenchant-dev

In [33]:
# !pip install distro

In [34]:
import distro
distro.linux_distribution()

('Amazon Linux AMI', '2018.03', '')

## nltk

In [35]:
# from nltk.corpus import words
# "would" in words.words()

In [36]:
# import nltk
# nltk.download('words')